<a href="https://colab.research.google.com/github/noahrubin989/Classification_Algorithms/blob/master/modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.3 MB/s 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np

import lightgbm
import catboost
import xgboost 

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

RANDOM_STATE = 42

In [ ]:
link = 'https://raw.githubusercontent.com/MillenniumForce/DATA3001/main/data/transformed_data.csv?token=GHSAT0AAAAAAB2IL7NATMWZY3GTDC3B7IQKY2WQ5KA'
df = pd.read_csv(link).drop(columns=['info'])

In [ ]:
def plot_confusion_matrix(final_model, X_test, y_test):
    yhat = final_model.predict(X_test)
    cm = confusion_matrix(y_test, yhat, labels=final_model.classes_)
    display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=final_model.classes_)
    display.plot();


def evaluate_classifier(clf):
    """Will add to this. Just print op=ut a whole lot of stuff then"""
    pass

In [ ]:
# Split data immediately

feature_matrix = df.drop(columns='target')
response_vector = df['target']

X_train, X_test, y_train, y_test = train_test_split(feature_matrix, 
                                                    response_vector, 
                                                    test_size=0.25, 
                                                    stratify=response_vector)

In [ ]:
# # Label encode the target
# label_encoder = LabelEncoder()

# y_train_new = label_encoder.fit_transform(y_train)
# y_test_new = label_encoder.transform(y_test)

# Random Forest

In [ ]:
params_rf = {
    'n_estimators':np.arange(100, 501, 50),
    'max_depth': np.arange(3, 17, 1),
    'min_samples_leaf': np.arange(3, 31, 2),
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=RANDOM_STATE)

search_rf = RandomizedSearchCV(estimator=rf, 
                               random_state=RANDOM_STATE,
                               param_distributions=params_rf, 
                               n_iter=10,  # 10 different (random) hyperparameter combos to try out
                               cv=5,  # run 5 fold cross validation on each random hyperparameter combination
                               # In the multi-class and multi-label case, this is the average of the F1 score of each class with weighting depending on the average parameter.
                               scoring='f1_weighted')

search_rf.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   param_distributions={'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]),
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': array([ 3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29]),
                                        'n_estimators': array([100, 150, 200, 250, 300, 350, 400, 450, 500])},
                   random_state=42, scoring='f1_weighted')

https://www.baeldung.com/cs/multi-class-f1-score

In [ ]:
best_rf_model = search_rf.best_estimator_  # It's already been fit

# one vs one based scoring
print(roc_auc_score(y_test, best_rf_model.predict_proba(X_test), multi_class='ovo'))

# one vs rest based scoring
print(roc_auc_score(y_test, best_rf_model.predict_proba(X_test), multi_class='ovr'))

0.911662559285691
0.9116809747694812


# LightGBM

In [ ]:
params_lgbm = {
    'eta': np.linspace(0.0001, 0.5, 100),
    'n_estimators': np.arange(100, 2001, 100),
    'max_depth': np.arange(3, 17, 1),
    'min_child_weight': np.arange(0, 25, 1),
    'subsample': np.linspace(0.8, 1, 20),
    'reg_alpha': [0.0001, 0.01, 0.1, 1, 10, 100],
    'colsample_bytree': [0.75, 0.8, 0.85, 0.9, 0.95, 1]
}

# Use gradient based one side sampling as well as exclusive feature bundling

# oops don't forget to add random state argument here!!
lgbm = lightgbm.LGBMClassifier(boosting='goss', enable_bundle=True)

search_lgbm = RandomizedSearchCV(estimator=lgbm,
                                 random_state=RANDOM_STATE,
                                 param_distributions=params_lgbm,
                                 n_iter=10,  # 10 different (random) hyperparameter combos to try out
                                 cv=5,  # run 5 fold cross validation on each random hyperparameter combination
                                 scoring='f1_weighted')

search_lgbm.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=LGBMClassifier(boosting='goss',
                                            enable_bundle=True),
                   param_distributions={'colsample_bytree': [0.75, 0.8, 0.85,
                                                             0.9, 0.95, 1],
                                        'eta': array([1.00000000e-04, 5.14949495e-03, 1.01989899e-02, 1.52484848e-02,
       2.02979798e-02, 2.53474747e-02, 3.03969697e-02, 3.54464646e-02,
       4.04959596e-02, 4.55454545e-02, 5.05949495e-02, 5.56444444e-02,
       6.069...
       1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000]),
                                        'reg_alpha': [0.0001, 0.01, 0.1, 1, 10,
                                                      100],
                                        'subsample': array([0.8       , 0.81052632, 0.82105263, 0.83157895, 0.84210526,
       0.85263158, 0.86315789, 0.87368421, 0.88421053, 0.89473684,
       0.90526316, 0.91578947

In [ ]:
best_lgbm_model = search_lgbm.best_estimator_

# one vs one based scoring
print(roc_auc_score(y_test, best_lgbm_model.predict_proba(X_test), multi_class='ovo'))

# one vs rest based scoring
print(roc_auc_score(y_test, best_lgbm_model.predict_proba(X_test), multi_class='ovr'))

0.9123036257254326
0.9123218853096157


# CatBoost (even though there are no categorical features LMAO)

* Nevertheless it stil performs well!
* Took so long to run when trying hyperparameter tuning that I'm just using the default params fmd

In [ ]:
catboost_model = catboost.CatBoostClassifier(random_state=RANDOM_STATE, loss_function='MultiClass')
catboost_model.fit(X_train, y_train)

Learning rate set to 0.093662
0:	learn: 2.3552782	total: 130ms	remaining: 2m 10s
1:	learn: 2.1784092	total: 201ms	remaining: 1m 40s
2:	learn: 2.0593857	total: 268ms	remaining: 1m 29s
3:	learn: 1.9608422	total: 338ms	remaining: 1m 24s
4:	learn: 1.8827482	total: 410ms	remaining: 1m 21s
5:	learn: 1.8207581	total: 475ms	remaining: 1m 18s
6:	learn: 1.7577461	total: 546ms	remaining: 1m 17s
7:	learn: 1.7033904	total: 619ms	remaining: 1m 16s
8:	learn: 1.6615173	total: 686ms	remaining: 1m 15s
9:	learn: 1.6239467	total: 754ms	remaining: 1m 14s
10:	learn: 1.5902549	total: 821ms	remaining: 1m 13s
11:	learn: 1.5591083	total: 897ms	remaining: 1m 13s
12:	learn: 1.5403946	total: 967ms	remaining: 1m 13s
13:	learn: 1.5140099	total: 1.04s	remaining: 1m 13s
14:	learn: 1.4862611	total: 1.11s	remaining: 1m 12s
15:	learn: 1.4669841	total: 1.17s	remaining: 1m 12s
16:	learn: 1.4496440	total: 1.25s	remaining: 1m 12s
17:	learn: 1.4402041	total: 1.31s	remaining: 1m 11s
18:	learn: 1.4208694	total: 1.38s	remaining:

In [ ]:
# one vs one based scoring
print(roc_auc_score(y_test, catboost_model.predict_proba(X_test), multi_class='ovo'))

# one vs rest based scoring
print(roc_auc_score(y_test, catboost_model.predict_proba(X_test), multi_class='ovr'))

0.9117844409703162
0.9118027280515122


In [ ]:
# # Will take years to run lol

# params_cb = {
#     'learning_rate': np.linspace(0.0001, 0.5, 100),
#     'n_estimators': np.arange(100, 501, 50),
#     'max_depth': np.arange(3, 17, 1),
#     'reg_lambda': [0.0001, 0.01, 0.1, 1, 10, 100]
# }

# cb = catboost.CatBoostClassifier(random_state=RANDOM_STATE, loss_function='MultiClass')

# search_cb = RandomizedSearchCV(estimator=cb,
#                                random_state=RANDOM_STATE,
#                                param_distributions=params_cb,
#                                n_iter=10,  # 10 different (random) hyperparameter combos to try out
#                                cv=5,  # run 5 fold cross validation on each random hyperparameter combination
#                                scoring='f1_weighted')

# search_cb.fit(X_train, y_train)

## Logistic Regression
* Will put this higher up the notebook since it is a baseline

In the multiclass case, the training algorithm uses the one-vs-rest (OvR) scheme if the ‘multi_class’ option is set to ‘ovr’, and uses the cross-entropy loss if the ‘multi_class’ option is set to ‘multinomial’. (Currently the ‘multinomial’ option is supported only by the ‘lbfgs’, ‘sag’, ‘saga’ and ‘newton-cg’ solvers.)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
scaler = StandardScaler()
logreg = LogisticRegression(random_state=RANDOM_STATE, max_iter=5000)

pipe = Pipeline(steps=[('scaler', scaler), ('logreg', logreg)])

params_lr = {
    'logreg__multi_class': ['auto', 'ovr', 'multinomial'],
}

search_lr = GridSearchCV(estimator=pipe, param_grid=params_lr, cv=5, scoring='f1_weighted')
search_lr.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('logreg',
                                        LogisticRegression(max_iter=5000,
                                                           random_state=42))]),
             param_grid={'logreg__multi_class': ['auto', 'ovr', 'multinomial']},
             scoring='f1_weighted')

In [ ]:
best_logreg_model = search_lr.best_estimator_
print(search_lr.best_params_)

# one vs one based scoring
print(roc_auc_score(y_test, best_logreg_model.predict_proba(X_test), multi_class='ovo'))

# one vs rest based scoring
print(roc_auc_score(y_test, best_logreg_model.predict_proba(X_test), multi_class='ovr'))

print('Model sucks lol')

{'logreg__multi_class': 'auto'}
0.7733544663287579
0.7733665376183326
Model sucks lol
